# Integración MAS-GC

## Introducción

En este  ejemplo, se tomó la implementación acerca de colisiones de cubos desde la perspectiva gráfica, creada por el Prof. Iván Olmos. A continuación, se muestra una adaptación de esta implementación, agregando la perspectiva multiagente.

### Estructura

Es importante mantener correctamente dividido la lógica que pertenece a los gráficos y, por otro lado, la de multiagentes. En este caso, se pretende que solamente lo correspondiente a dibujo, le corresponde a gráficos, mientras que los mecanísmos de control (velocidad, dirfección, colisión, decisión, etc.) son parte de sistemas multiagentes.

Se usa PyGame para la parte gráfica, y AgentPy para la parte de agentes.

Se divide el programa en tres partes: Cubo, PlanosCubo, Modelo. El primero contiene lo necesario para dibujar cada instancia de cubo. El segundo tiene lo necesario para dibujar el ambiente (en este caso, planos y ejes) (aquí no se dibuja ninguna instancia de cualquier otro objeto). Finalmente, Modelo tiene el código acerca de la simulación de agentes, y que hace uso de las demás partes para crear las instancias gráficas.

## Planos Cubos

Similar al programa original. Se eliminan las partes que crean cubos en el mundo gráfico. También se elimina la variables que especifíca el tamaño del plano (DimBoard).

### Importaciones de módulos

In [ ]:
# Importaciones: PyGame y OpenGL
import pygame
from pygame.locals import *

# Cargamos las bibliotecas de OpenGL
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

### Variables gráficas globales

Aquí se omite la variable de dimensiones del plano (DimBoard)

In [ ]:
screen_width = 500
screen_height = 500
#vc para el obser.
FOVY=60.0
ZNEAR=0.01
ZFAR=900.0
#Variables para definir la posicion del observador
#gluLookAt(EYE_X,EYE_Y,EYE_Z,CENTER_X,CENTER_Y,CENTER_Z,UP_X,UP_Y,UP_Z)
EYE_X=300.0
EYE_Y=200.0
EYE_Z=300.0
CENTER_X=0
CENTER_Y=0
CENTER_Z=0
UP_X=0
UP_Y=1
UP_Z=0
#Variables para dibujar los ejes del sistema
X_MIN=-500
X_MAX=500
Y_MIN=-500
Y_MAX=500
Z_MIN=-500
Z_MAX=500

### Inicialización de PyGame

In [ ]:
pygame.init()

### Definición de dibujado de Ejes

In [ ]:
def Axis():
    glShadeModel(GL_FLAT)
    glLineWidth(3.0)
    #X axis in red
    glColor3f(1.0,0.0,0.0)
    glBegin(GL_LINES)
    glVertex3f(X_MIN,0.0,0.0)
    glVertex3f(X_MAX,0.0,0.0)
    glEnd()
    #Y axis in green
    glColor3f(0.0,1.0,0.0)
    glBegin(GL_LINES)
    glVertex3f(0.0,Y_MIN,0.0)
    glVertex3f(0.0,Y_MAX,0.0)
    glEnd()
    #Z axis in blue
    glColor3f(0.0,0.0,1.0)
    glBegin(GL_LINES)
    glVertex3f(0.0,0.0,Z_MIN)
    glVertex3f(0.0,0.0,Z_MAX)
    glEnd()
    glLineWidth(1.0)

### Inicialización de mundo gráfico

In [ ]:
def Init():
    screen = pygame.display.set_mode(
        (screen_width, screen_height), DOUBLEBUF | OPENGL)
    pygame.display.set_caption("OpenGL: cubos")

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(FOVY, screen_width/screen_height, ZNEAR, ZFAR)

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    gluLookAt(EYE_X,EYE_Y,EYE_Z,CENTER_X,CENTER_Y,CENTER_Z,UP_X,UP_Y,UP_Z)
    glClearColor(0,0,0,0)
    glEnable(GL_DEPTH_TEST)
    glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)

### Función para dibujado del mundo gráfico

En este parte se omite el dibujado de instancias de cubos. Solo se dibujan los ejes y el plano.

In [ ]:
ef display(DimBoard):
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    Axis()
    #Se dibuja el plano gris
    glColor3f(0.3, 0.3, 0.3)
    glBegin(GL_QUADS)
    glVertex3d(-DimBoard, 0, -DimBoard)
    glVertex3d(-DimBoard, 0, DimBoard)
    glVertex3d(DimBoard, 0, DimBoard)
    glVertex3d(DimBoard, 0, -DimBoard)
    glEnd()

## Cubo

Similar al programa original. Se eliminan las partes que contienen mecanismos de control para las trayectorias de los cubos. También se omiten lo relacionado al mecanismo de detección de colisión (incluyendo listas de instancias de cubos).

### Importaciones de módulos

In [ ]:
# Importaciones: Pygame, Opengl, y herramientas necesarias
import pygame
from pygame.locals import *

# Cargamos las bibliotecas de OpenGL
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

import numpy as np
import random
import math

### Clase Cubo

Aquí se agregan métodos para inicializar, dibujado y trazado del cubo.

In [ ]:
class Cubo:

    # Solo se inicializa una positión inicial, así como los puntos del cubo
    def __init__(self, init_pos=(0,0,0)):
        self.points = np.array([[-1.0,-1.0, 1.0], [1.0,-1.0, 1.0], [1.0,-1.0,-1.0], [-1.0,-1.0,-1.0],
                                [-1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,-1.0], [-1.0, 1.0,-1.0]])

        self.Position = list(init_pos)

    # Dibujado de las caras del cubo
    def drawFaces(self):
        glBegin(GL_QUADS)
        glVertex3fv(self.points[0])
        glVertex3fv(self.points[1])
        glVertex3fv(self.points[2])
        glVertex3fv(self.points[3])
        glEnd()
        glBegin(GL_QUADS)
        glVertex3fv(self.points[4])
        glVertex3fv(self.points[5])
        glVertex3fv(self.points[6])
        glVertex3fv(self.points[7])
        glEnd()
        glBegin(GL_QUADS)
        glVertex3fv(self.points[0])
        glVertex3fv(self.points[1])
        glVertex3fv(self.points[5])
        glVertex3fv(self.points[4])
        glEnd()
        glBegin(GL_QUADS)
        glVertex3fv(self.points[1])
        glVertex3fv(self.points[2])
        glVertex3fv(self.points[6])
        glVertex3fv(self.points[5])
        glEnd()
        glBegin(GL_QUADS)
        glVertex3fv(self.points[2])
        glVertex3fv(self.points[3])
        glVertex3fv(self.points[7])
        glVertex3fv(self.points[6])
        glEnd()
        glBegin(GL_QUADS)
        glVertex3fv(self.points[3])
        glVertex3fv(self.points[0])
        glVertex3fv(self.points[4])
        glVertex3fv(self.points[7])
        glEnd()

    # dibujado del cubo
    def draw(self, Position, scale):
        glPushMatrix()
        glTranslatef(Position[0], Position[1], Position[2])
        glScaled(scale,scale,scale)
        glColor3f(1.0, 1.0, 1.0)
        self.drawFaces()
        glPopMatrix()




## Modelo

Aquí se utilizan los módulos anteriores para instancias gráficamente los cubos de cada agente un el mundo 3D

### Importaciones

Se importa el framework de AgentPy, y los dos módulos construidos anteriormente.

In [ ]:
import agentpy as ap
import Cubo
import PlanoCubos
import math
import random
import pygame
# importando un motor para presentar gráficas
import matplotlib.pyplot as plt

## CuboAgent

In [ ]:
class CuboAgent(ap.Agent):

    def setup(self):
        # iniciualización de variables
        self.vel = self.model.p.vel
        self.scale = self.model.p.Scale
        self.radio = math.sqrt(self.scale*self.scale + self.scale*self.scale)
        self.DimBoard = self.model.p.dim
        #Se inicializa una posicion aleatoria en el tablero
        self.Position = []
        self.Position.append(random.randint(-1 * self.DimBoard, self.DimBoard))
        self.Position.append(self.scale)
        self.Position.append(random.randint(-1 * self.DimBoard, self.DimBoard))
        #Se inicializa un vector de direccion aleatorio
        self.Direction = []
        self.Direction.append(random.random())
        self.Direction.append(self.scale)
        self.Direction.append(random.random())
        #Se normaliza el vector de direccion
        m = math.sqrt(self.Direction[0]*self.Direction[0] + self.Direction[2]*self.Direction[2])
        self.Direction[0] /= m
        self.Direction[2] /= m
        #Se cambia la maginitud del vector direccion
        self.Direction[0] *= self.vel
        self.Direction[2] *= self.vel
        #deteccion de colision
        self.collision = False

        # instanciar la representación gráfica de cubo
        self.g_cubo = Cubo.Cubo(self.Position)
        self.g_cubo.draw(self.Position,self.scale)
        pass

    # Cada iteración  se detectan colisiones, si hay una colisión, entonces
    # se invierte la dirección (el cubo rebota)
    def step(self):
        self.collision = False
        self.CollitionDetection()
        if self.collision == False:
            new_x = self.Position[0] + self.Direction[0]
            new_z = self.Position[2] + self.Direction[2]
        else:
            self.Direction[2] *= -1.0
            self.Direction[0] *= -1.0
            new_x = self.Position[0] + self.Direction[0]
            new_z = self.Position[2] + self.Direction[2]

        # Se comprueba que los cubos estén dentro del plano
        if(abs(new_x) <= self.DimBoard):
            self.Position[0] = new_x
        else:
            self.Direction[0] *= -1.0
            self.Position[0] += self.Direction[0]

        if(abs(new_z) <= self.DimBoard):
            self.Position[2] = new_z
        else:
            self.Direction[2] *= -1.0
            self.Position[2] += self.Direction[2]
        pass

    # En el update se hace el dibujado de la instancia gráfica del cubo
    def update(self):
        self.g_cubo.draw(self.Position, self.scale)

    # método para detección de colisiones
    def CollitionDetection(self):
        # Por cada agente en el modelo...
        for ag in self.model.cubos:
            # Si el id del agente no es el mismo...
            if self.id != ag.id:
                # Calcular distancias (con todo y colider)
                d_x = self.Position[0] - ag.Position[0]
                d_z = self.Position[2] - ag.Position[2]
                d_c = math.sqrt(d_x * d_x + d_z * d_z)
                if d_c - (self.radio + ag.radio) < 0.0:
                    self.collision = True
                    # Más uno a la cantidad de colisiones por step
                    self.model.collisions += 1

### Cubo Model

In [ ]:
class CuboModel(ap.Model):

    def setup(self):
        # Inicialización de Agentes
        self.cubos = ap.AgentList(self,self.p.cubos,CuboAgent)
        self.collisions = 0 # Contador de colisiones
        pass

    def step(self):
        # Llamar a cada step de los agentes
        self.cubos.step()
        pass

    def update(self):
        # Dibujado de cada agente
        self.cubos.update()
        # Registro de datos: Cantidad de colisiones por step
        self.record('Cantidad de colisiones', self.collisions)
        self.collisions = 0 # Reinicio de contador
        pass

    def end(self):
        pass

## **Main**

In [ ]:
# Definición de parámetros de la simulación

parameters = {
   'cubos' : 100,
   'dim' : 200,
   'vel' : 2.0,
   'Scale' : 5.0,
   #'steps' : 100
}

model = CuboModel(parameters) # instanciación de Cubo Model

done = False # Flag de detención de la app
PlanoCubos.Init() # Inicialización del mundo gráfico
model.sim_setup() # Inicialización manual de Simulación

# Ciclo principal
while not done:
    # Si se ha cerrado la aplicación, activar flags
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True # Flag de dentención
            model.stop() # Llamar a detención de simulación manualmente
            model.create_output() # Crear registro manualmente
            # Agregar información al registro postSimulación
            model.output.info['Mensaje'] = 'Puedes añadir información al registro de esta forma.'

    # Dibujado del mundo gráfico
    PlanoCubos.display(parameters['dim'])

    # Si la simulación está corriendo...
    if model.running:
        # Llamar a la iteración de la simulación manualmente
        model.sim_step()


    pygame.display.flip()
    pygame.time.wait(10)

pygame.quit() # Finalizar entorno gráfico

print(model.output.info)
# Graficar variable registrada
model.output.variables.CuboModel.plot()
plt.show()